In [5]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pydicom
import nibabel as nib
import cv2

# ----------------------------
# 1. SCAN LOADING & PREPROCESSING
# ----------------------------
import pathlib

def load_scan(file_path):
    import pathlib
    file_path = str(pathlib.Path(file_path))

    if file_path.endswith('.nii') or file_path.endswith('.nii.gz'):
        img = nib.load(file_path)
        return img.get_fdata()
    elif os.path.isdir(file_path):
        slices = [pydicom.dcmread(os.path.join(file_path, f)) for f in os.listdir(file_path) if f.endswith('.dcm')]
        slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))
        volume = np.stack([s.pixel_array for s in slices], axis=-1)
        return volume
    else:
        raise ValueError(f"Unsupported format or bad path: {file_path}")

def normalize_volume(volume):
    volume = volume.astype(np.float32)
    volume -= np.min(volume)
    volume /= np.max(volume)
    return volume

def extract_12_slices(volume):
    slices = []
    z_len = volume.shape[2]
    axial_indices = np.linspace(0, z_len - 1, 10, dtype=int)
    axial_slices = [volume[:, :, idx] for idx in axial_indices]

    coronal_idx = volume.shape[1] // 2
    coronal_slice = volume[:, coronal_idx, :]

    sagittal_idx = volume.shape[0] // 2
    sagittal_slice = volume[sagittal_idx, :, :]

    slices.extend(axial_slices)
    slices.append(coronal_slice)
    slices.append(sagittal_slice)

    return slices

def preprocess_scan(path, target_size=(224, 224)):
    volume = load_scan(path)
    volume = normalize_volume(volume)
    slices = extract_12_slices(volume)
    resized = [cv2.resize(s, target_size, interpolation=cv2.INTER_AREA) for s in slices]
    return np.stack(resized, axis=0)  # (12, 224, 224)

# ----------------------------
# 2. DATASET CLASS
# ----------------------------
class CTADataset(Dataset):
    def __init__(self, labels_csv, root_dir, transform=None):
        self.data = pd.read_csv(labels_csv)
        self.root_dir = r"C:\Users\HenryLi\Downloads\Scans"
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data.iloc[idx]
        filename = entry['filename']
    
    # Assume that scan files live in: {root_dir}/{scan_id}/20/{filename}
        scan_id = filename.split('-')[1].split()[0]  # e.g. '11001'
        file_path = os.path.join(self.root_dir, scan_id, '20', filename)

        slices = preprocess_scan(file_path)
        slices = torch.tensor(slices).unsqueeze(1).float()

        if self.transform:
            slices = self.transform(slices)

        label = torch.tensor(entry['label']).float()
        return slices, label

# ----------------------------
# 3. MODEL DEFINITION (2D CNN + Aggregation)
# ----------------------------
class SliceCNN(nn.Module):
    def __init__(self, feature_dim=128):
        super(SliceCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.fc = nn.Linear(128, feature_dim)

    def forward(self, x):  # (B, 1, 224, 224)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

class CTAQuality2D(nn.Module):
    def __init__(self, feature_dim=128):
        super(CTAQuality2D, self).__init__()
        self.slice_cnn = SliceCNN(feature_dim)
        self.regressor = nn.Sequential(
            nn.Linear(feature_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):  # (B, 12, 1, 224, 224)
        B, S, C, H, W = x.shape
        x = x.view(B * S, C, H, W)
        feats = self.slice_cnn(x)  # (B×12, F)
        feats = feats.view(B, S, -1)
        pooled = feats.mean(dim=1)
        out = self.regressor(pooled)
        return out.squeeze(1)

# ----------------------------
# 4. TRAINING AND EVALUATION LOOPS
# ----------------------------
def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    preds, targets = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * inputs.size(0)
            preds.append(outputs.cpu())
            targets.append(labels.cpu())
    preds = torch.cat(preds)
    targets = torch.cat(targets)
    return total_loss / len(dataloader.dataset), preds, targets


# ----------------------------
# 5. TRAINING DRIVER CODE
# ----------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

# Settings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
root_dir = 'root_dir = "C:\\Users\\HenryLi\\Downloads\\Scans"'
labels_csv = "C:/Users/HenryLi/Desktop/Python Projects/Primitive Image Classifier/Labels.csv"
batch_size = 8
num_epochs = 30

# Load and split CSV
df = pd.read_csv(labels_csv)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)

train_loader = DataLoader(CTADataset('train.csv', root_dir), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(CTADataset('val.csv', root_dir), batch_size=batch_size)

model = CTAQuality2D().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

best_val_loss = float('inf')
for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_preds, val_targets = evaluate(model, val_loader, criterion)
    scheduler.step()

    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pt')

    r2 = r2_score(val_targets.numpy(), val_preds.numpy())
    mae = mean_absolute_error(val_targets.numpy(), val_preds.numpy())
    print(f"Val R^2: {r2:.3f}, MAE: {mae:.3f}")

#Inspecting predictions
print("Sample Predictions vs Targets:")
for pred, target in zip(val_preds[:10], val_targets[:10]):
    print(f"Pred: {pred.item():.2f}, Target: {target.item():.2f}")


Epoch 1: Train Loss=5.9466, Val Loss=5.9471
Val R^2: -3.662, MAE: 2.161
Epoch 2: Train Loss=5.4033, Val Loss=5.0863
Val R^2: -2.987, MAE: 1.948
Epoch 3: Train Loss=3.9929, Val Loss=2.8160
Val R^2: -1.207, MAE: 1.319
Epoch 4: Train Loss=1.8490, Val Loss=1.6317
Val R^2: -0.279, MAE: 1.096
Epoch 5: Train Loss=1.3640, Val Loss=1.5043
Val R^2: -0.179, MAE: 1.052
Epoch 6: Train Loss=1.3049, Val Loss=1.4954
Val R^2: -0.172, MAE: 1.051
Epoch 7: Train Loss=1.3240, Val Loss=1.4937
Val R^2: -0.171, MAE: 1.050
Epoch 8: Train Loss=1.2987, Val Loss=1.4796
Val R^2: -0.160, MAE: 1.043
Epoch 9: Train Loss=1.3011, Val Loss=1.4707
Val R^2: -0.153, MAE: 1.040
Epoch 10: Train Loss=1.2832, Val Loss=1.4639
Val R^2: -0.147, MAE: 1.038


KeyboardInterrupt: 

## Interpretation of Results
This is for the 1st Epoch
| Metric       | Value     | Meaning                                                                 |
|--------------|-----------|-------------------------------------------------------------------------|
| **Train Loss** | 5.9466   | MSE on training data — higher means worse predictions                   |
| **Val Loss**   | 5.9471   | MSE on validation — similar to train loss suggests underfitting         |
| **R² (Val)**   | -3.662   | Negative R² → model is worse than predicting the mean                   |
| **MAE**        | 2.161    | Average absolute error is ~2 points (on a 1–5 scale) — too high         |